<a href="https://colab.research.google.com/github/XORbit01/IHCA_data_cleaning/blob/main/IHCA_DATA_CLEANING_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IHCA Data Cleaning Workflow

This notebook chronicles the end-to-end cleaning pipeline for the in-hospital cardiac arrest (IHCA) registry. The intent is to:

- consolidate records captured in the `ND` and `ICU` source tabs
- standardize clinical variables so downstream analytics receive consistent schemas
- cross-check event timelines for internal consistency
- derive interpretable features needed for survival and resuscitation analyses

Each section states the clinical or data-quality motivation behind the transformations so that peers can audit, reproduce, or extend the workflow with confidence.


## 1. Environment Setup

Install and import the core libraries required for the cleaning workflow. The dataset manipulation relies on `pandas` for structured ingests and vectorized transformations.


In [ ]:
%pip install pandas

### 2. Load Source Workbooks

Read both source tabs (`ND` and `ICU`) from the master Excel file. Keeping them in separate DataFrames allows us to compare schemas and diagnose inconsistencies before merging.


In [ ]:
import pandas as pd

df_nd = pd.read_excel("IHCA data 00.xlsx", sheet_name="ND")
df_icu = pd.read_excel("IHCA data 00.xlsx", sheet_name="ICU")

### 3. Profile Raw Schemas

Inspect column names, data types, and non-null counts for each sheet. Early profiling highlights naming drift and missingness patterns that must be reconciled before concatenating the sources.


In [ ]:
# Review schema for the nursing dataset tab
df_nd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   MRN                                200 non-null    int64         
 1   Gend                               200 non-null    object        
 2   Birth
Year                         200 non-null    int64         
 3   Arrest
Year                        200 non-null    int64         
 4   Age                                200 non-null    int64         
 5   CAD                                200 non-null    object        
 6   Heart
failure                      200 non-null    object        
 7   Heart 
disease                     200 non-null    object        
 8   Hyper
-tension                     200 non-null    object        
 9   COPD                               200 non-null    object        
 10  Diabetes                           200

In [ ]:
df_icu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673 entries, 0 to 672
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   MRN                                673 non-null    int64         
 1   Gend.                              672 non-null    object        
 2   Birth
year                         672 non-null    float64       
 3   Arrest 
year                       673 non-null    int64         
 4   Age                                672 non-null    float64       
 5   CAD                                673 non-null    object        
 6   Heart
failure                      673 non-null    object        
 7   Heart 
disease                     673 non-null    object        
 8   Hyper
-tension                     672 non-null    object        
 9   COPD                               673 non-null    object        
 10  Diabetes                           673

### 4. Align Column Names

A helper function surfaces naming mismatches between tabs. After normalizing case, whitespace, and punctuation we confirm the schemas align before unioning the records.


In [ ]:
def difference():
    """Report column names that exist in only one of the source DataFrames."""
    diff_nd = df_nd.columns.difference(df_icu.columns)
    diff_icu = df_icu.columns.difference(df_nd.columns)

    print("Column names in df_nd but not in df_icu:")
    display(diff_nd)

    print("\nColumn names in df_icu but not in df_nd:")
    display(diff_icu)

In [ ]:
# Normalize header casing and remove stray whitespace/newline characters
import re

df_nd.columns = df_nd.columns.str.lower().str.replace('\n', ' ').str.strip()
df_icu.columns = df_icu.columns.str.lower().str.replace('\n', ' ').str.strip()

# Collapse repeated spaces and strip trailing punctuation to align naming conventions
df_nd.columns = (
    df_nd.columns
        .str.replace(r'\.+$', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)

df_icu.columns = (
    df_icu.columns
        .str.replace(r'\.+$', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)

In [ ]:
# check the difference again
difference()

Column names in df_nd but not in df_icu:


Index(['event loc'], dtype='object')


Column names in df_icu but not in df_nd:


Index(['event location'], dtype='object')

### 5. Merge Labeled Sources

Tag each record with its origin (`ND` vs `ICU`) so provenance remains traceable after concatenation. Column renaming consolidates verbose headers into analytics-friendly field names while preserving clinical meaning.


In [ ]:
# Harmonize the event location header before concatenation
df_nd.rename(columns={'event loc': 'event_location'}, inplace=True)
df_icu.rename(columns={'event location': 'event_location'}, inplace=True)

In [ ]:
difference()

Column names in df_nd but not in df_icu:


Index([], dtype='object')


Column names in df_icu but not in df_nd:


Index([], dtype='object')

In [ ]:
# Preserve provenance while combining both source tabs
df_nd["origin"] = "ND"
df_icu["origin"] = "ICU"
df = pd.concat([df_nd, df_icu], ignore_index=True)
df.columns

Index(['mrn', 'gend', 'birth year', 'arrest year', 'age', 'cad',
       'heart failure', 'heart disease', 'hyper -tension', 'copd', 'diabetes',
       'cancer', 'covid at adm', 'smoking', 'event_location', 'arrest date',
       'arrest time', 'cpr date', 'cpr time',
       'defibrillation (time, frequency)', 'initial cardiac rhythm',
       'rosc (y/n)', 'rosc date', 'rosc time', '24 hours survival',
       'survival to discharge', 'origin'],
      dtype='object')

In [ ]:
# Rename verbose fields to modeling-friendly aliases while retaining meaning
df = df.rename(columns={
    'mrn': 'mrn',
    'gend': 'gender',
    'birth year': 'birth_year',
    'arrest year': 'arrest_year',
    'age': 'age',
    'cad': 'coronary_artery_disease',
    'heart failure': 'heart_failure',
    'heart disease': 'heart_disease',
    'hyper -tension': 'hypertension',
    'copd': 'copd',
    'diabetes': 'diabetes',
    'cancer': 'cancer',
    'covid at adm': 'covid_on_admission',
    'smoking': 'smoking',
    'event_location': 'event_location',
    'arrest date': 'arrest_date',
    'arrest time': 'arrest_time',
    'cpr date': 'cpr_date',
    'cpr time': 'cpr_time',
    'defibrillation (time, frequency)': 'defibrillation_info',
    'initial cardiac rhythm': 'initial_rhythm',
    'rosc (y/n)': 'rosc',
    'rosc date': 'rosc_date',
    'rosc time': 'rosc_time',
    '24 hours survival': 'survival_24h',
    'survival to discharge': 'survival_to_discharge',
    'origin': 'origin'
})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   mrn                      873 non-null    int64         
 1   gender                   872 non-null    object        
 2   birth_year               872 non-null    float64       
 3   arrest_year              873 non-null    int64         
 4   age                      872 non-null    float64       
 5   coronary_artery_disease  873 non-null    object        
 6   heart_failure            873 non-null    object        
 7   heart_disease            873 non-null    object        
 8   hypertension             872 non-null    object        
 9   copd                     873 non-null    object        
 10  diabetes                 873 non-null    object        
 11  cancer                   873 non-null    object        
 12  covid_on_admission       873 non-nul

### 6. Create a Working Copy for Feature Standardization

Perform exploratory validation and cleaning on a copy of the merged dataset (`df_eda`). This ensures the raw merged frame remains unchanged for auditing while we iteratively standardize each domain.


In [ ]:
# Preserve the merged dataset and work off a mutable copy
df_eda = df.copy()


#### 6.1 Standardize Demographic Fields

Validate the `gender` entries to ensure no missing values remain and that categorical encodings are explicit (`M`, `F`, `U`).


In [ ]:
# Inspect the raw gender distribution and missingness metrics
df_eda["gender"].value_counts(dropna=False)
df_eda["gender"].unique()
df_eda["gender"].isna().sum()


np.int64(1)

In [ ]:
# Impute missing gender values with 'U' (Unknown) for downstream consistency
df_eda["gender"] = df_eda["gender"].fillna("U")

#### 6.2 Validate Age Consistency

Confirm age values remain physiologically plausible by triangulating `age`, `birth_year`, and `arrest_year`. This guards against transcription errors that could bias risk models.


In [ ]:
# Review descriptive statistics for the age field
_df_age_summary = df["age"].describe()
_df_age_summary

,age
count,872.000000
mean,71.147936
std,19.964950
min,0.000000
25%,64.000000
50%,76.000000
75%,85.000000
max,113.000000


In [ ]:
# Quantify neonate/infant cases (age ≤ 1 year)
(df["age"] <= 1).sum()

np.int64(15)

In [ ]:
# Cross-check reported age against birth and arrest years to flag transcription issues
df_eda["computed_age"] = df_eda["arrest_year"] - df_eda["birth_year"]

df_eda["age_mismatch"] = (df_eda["age"] - df_eda["computed_age"]).abs() > 1

df_eda.loc[df_eda["age_mismatch"], ["age", "birth_year", "arrest_year", "computed_age"]]


,age,birth_year,arrest_year,computed_age


In [ ]:
# Remove helper flag after confirming no discrepancies
df_eda = df_eda.drop(columns=["age_mismatch"])

In [ ]:
# Validation summary: neonatal records align with arrest and birth years.
# The observed minimum age represents true infant cases rather than data entry errors.

In [ ]:
df_eda = df_eda.drop(columns=["computed_age"])  # Remove temporary calculation column

#### 6.3 Standardize Chronic Condition Flags

Iteratively validate each comorbidity indicator, harmonizing free-text values and resolving implausible combinations across cardiovascular diagnoses.


In [ ]:
# Explore raw hypertension responses to scope normalization rules
df_eda["hypertension"].value_counts(dropna=False)


,count
hypertension,
yes,535
no,337
NaN,1


In [ ]:
# Harmonize hypertension values to a consistent Y/N/U encoding
df_eda["hypertension"] = df_eda["hypertension"].replace({"yes": "Y", "no": "N"})
df_eda["hypertension"] = df_eda["hypertension"].fillna("U")

In [ ]:
# Review diabetes responses before normalization
df_eda["diabetes"].value_counts(dropna=False)

,count
diabetes,
no,476
yes,391
Yes,3
No,2
na,1


In [ ]:
# Map free-text diabetes responses to canonical categories
df_eda["diabetes"] = df_eda["diabetes"].replace({
    "yes": "Y",
    "Yes": "Y",
    "no": "N",
    "No": "N",
    "na": "U"
})

# Convert genuine NaN values to Unknown
df_eda["diabetes"] = df_eda["diabetes"].fillna("U")


In [ ]:
# Review heart_failure responses before standardization
df_eda["heart_failure"].value_counts(dropna=False)


,count
heart_failure,
no,699
yes,174


In [ ]:
# Normalize heart_failure entries to Y/N/U
df_eda["heart_failure"] = df_eda["heart_failure"].replace({
    "yes": "Y",
    "no": "N",
})

df_eda["heart_failure"] = df_eda["heart_failure"].fillna("U")


In [ ]:
# Review coronary_artery_disease responses before standardization
df_eda["coronary_artery_disease"].value_counts(dropna=False)


,count
coronary_artery_disease,
no,848
yes,24
yn,1


In [ ]:
# Treat ambiguous 'yn' entries as Unknown to avoid overstating prevalence

In [ ]:
df_eda["coronary_artery_disease"] = df_eda["coronary_artery_disease"].replace({
    "yes": "Y",
    "no": "N",
    "yn": "U"   # suspicious → mark as Unknown
})

df_eda["coronary_artery_disease"] = df_eda["coronary_artery_disease"].fillna("U")


In [ ]:
# Inspect heart_disease responses and align with Y/N conventions
df_eda["heart_disease"].value_counts(dropna=False)
df_eda["heart_disease"] = df_eda["heart_disease"].replace({
    "yes": "Y",
    "no": "N",
})


In [ ]:
# Identify records where CAD and heart failure are both positive but heart_disease is negative
# Clinically, such combinations likely indicate an entry error.
df_eda["heart_disease_mismatch"] = (
    (df_eda["coronary_artery_disease"] == "Y") &
    (df_eda["heart_failure"] == "Y") &
    (df_eda["heart_disease"] == "N")
)
df_eda["heart_disease_mismatch"].value_counts()

,count
heart_disease_mismatch,
False,871
True,2


In [ ]:
# Review the conflicting rows to confirm they appear to be data-entry issues rather than true exceptions
df_eda[
    df_eda["heart_disease_mismatch"]
][[
    "age",
    "gender",
    "coronary_artery_disease",
    "heart_failure",
    "heart_disease",
    "smoking",
    "diabetes",
    "hypertension",
    "copd",
    "cancer",
    "survival_to_discharge",
    "rosc",
    "origin"
]]


,age,gender,coronary_artery_disease,heart_failure,heart_disease,smoking,diabetes,hypertension,copd,cancer,survival_to_discharge,rosc,origin
708,77.0,M,Y,Y,N,yes,Y,Y,yes,no,no,yes,ICU
731,59.0,F,Y,Y,N,na,Y,Y,no,no,no,yes,ICU


In [ ]:
# Override the two conflicting records so that severe cardiac diagnoses remain internally consistent
df_eda.loc[
    (df_eda["coronary_artery_disease"] == "Y") |
    (df_eda["heart_failure"] == "Y"),
    "heart_disease"
] = "Y"


In [ ]:
# Remove the temporary mismatch indicator after resolving discrepancies
df_eda = df_eda.drop(columns=["heart_disease_mismatch"])


In [ ]:
# Inspect COPD field for inconsistent encodings
df_eda["copd"].value_counts(dropna=False)

,count
copd,
no,786
yes,87


In [ ]:
# Standardize COPD responses to uppercase categorical flags
df_eda["copd"] = df_eda["copd"].replace({
    "yes": "Y", "Yes": "Y",
    "no": "N", "No": "N",
    "na": "U", "Na": "U", "NA": "U"
})


In [ ]:
# Inspect cancer responses before recoding
df_eda["cancer"].value_counts(dropna=False)

,count
cancer,
no,752
yes,121


In [ ]:
# Values appear standardized but will still map to ensure case consistency

In [ ]:
# Standardize cancer responses to Y/N
df_eda["cancer"] = df_eda["cancer"].replace({
    "yes": "Y", "Yes": "Y",
    "no": "N", "No": "N",
})

In [ ]:
# Inspect smoking status values to design canonical mapping
df_eda["smoking"].value_counts(dropna=False)

,count
smoking,
no,661
yes,191
na,10
No,3
Yes,2
NaN,2
NO,1
old,1
previous,1


In [ ]:
# Map smoking history to current (`Y`), none (`N`), previous (`P`), or unknown (`U`)
df_eda["smoking"] = df_eda["smoking"].replace({
    "yes": "Y",
    "Yes": "Y",
    "YES": "Y",

    "no": "N",
    "No": "N",
    "NO": "N",

    "old": "Y",
    "previous": "Y",
    "Previous": "Y",
})

df_eda["smoking"] = df_eda["smoking"].fillna("U")


In [ ]:
# Validate the distribution after recoding
df_eda["smoking"].value_counts(dropna=False)


,count
smoking,
N,665
Y,195
na,10
U,2
yes,1


In [ ]:
# Trim whitespace artifacts that may have prevented replacements from applying
df_eda["smoking"] = df_eda["smoking"].str.strip()

In [ ]:
# Re-run mapping after whitespace cleanup to capture remaining variants
df_eda["smoking"] = df_eda["smoking"].replace({
    "yes": "Y",
    "Yes": "Y",
    "YES": "Y",

    "no": "N",
    "No": "N",
    "NO": "N",
    "na": "U",
    "old": "P",
    "previous": "P",
    "Previous": "P",
})

df_eda["smoking"] = df_eda["smoking"].fillna("U")


In [ ]:
# Confirm final smoking distribution and residual unknown volume
df_eda["smoking"].value_counts(dropna=False)


,count
smoking,
N,665
Y,196
U,12


In [ ]:
# Note: 12 encounters remain Unknown; retain for now pending clinical clarification

#### 6.4 Admission Infection Status

Standardize the COVID-on-admission indicator to the core `Y/N` vocabulary used across the dataset.


In [ ]:
# Review COVID-on-admission responses prior to recoding
df_eda["covid_on_admission"].value_counts(dropna=False)


,count
covid_on_admission,
no,808
yes,65


In [ ]:
# Normalize admission infection status to consistent uppercase codes
df_eda["covid_on_admission"] = df_eda["covid_on_admission"].replace({
    "yes": "Y",
    "no": "N",
})

#### 6.5 Event Context & Timing Variables

Normalize the location codes and validate the sequence of arrest, CPR, and ROSC timestamps to ensure temporal consistency.


In [ ]:
# 6.5 Event Context & Timing Variables
# Normalize the location codes and validate the sequence of arrest, CPR, and ROSC timestamps to ensure temporal consistency.

In [ ]:
df_eda["event_location"].value_counts(dropna=False)

,count
event_location,
ICU,638
MMS,69
CVU,62
FMS,35
CCU,22
ped,9
DSU,9
covid,7
POST ICU,6


**Operational definitions for `event_location`:**

| Code | Interpretation |
| --- | --- |
| ICU, ICU2, POST ICU, CCU, CVU, PICU, NICU | Intensive or cardiac critical care units |
| MMS, FMS, COVID, COVID UNIT | Medical/surgical wards (grouped as ward care) |
| PED, PEDIATRIC | Dedicated pediatric ward |
| DSU, DSU (acineto), OPERATING ROOM | Procedural or perioperative spaces |
| OBS | Obstetrics ward |
| NA | Missing/unknown location |


In [ ]:
# Reference the glossary above when interpreting event location codes

In [ ]:
# Normalize casing and remove trailing whitespace before recoding
df_eda["event_location"] = df_eda["event_location"].str.lower().str.strip()

In [ ]:
# Consolidate synonymous unit labels prior to classification
df_eda["event_location"] = df_eda["event_location"].replace({
    "icu2": "icu",
    "post icu": "post_icu",
    "covid": "covid_unit",
    "dsu (acineto)": "dsu",
    "ped": "pediatric",
    "picu": "pediatric_icu",
    "nicu": "neonatal_icu",
    "or": "operating_room",
    "obs": "obstetrics"
})


In [ ]:
# Rename remaining COVID isolation labels to a single value
df_eda["event_location"] = df_eda["event_location"].replace({
    "covid_unit": "covid_ward"
})

In [ ]:
df_eda["event_location"] = df_eda["event_location"].replace("na", "unknown")  # Flag missing units explicitly

In [ ]:
# Verify final location distribution after normalization
df_eda["event_location"].value_counts(dropna=False)


,count
event_location,
icu,640
mms,69
cvu,62
fms,35
ccu,22
pediatric,12
covid_ward,10
dsu,10
post_icu,6


In [ ]:
# Inspect arrest, CPR, and ROSC dates before conversion
df_eda[["arrest_date", "cpr_date", "rosc_date"]]


,arrest_date,cpr_date,rosc_date
0,2018-06-26,2018-06-26 00:00:00,2018-06-26 00:00:00
1,2018-07-14,2018-07-14 00:00:00,2018-07-14 00:00:00
2,2018-07-30,2018-07-30 00:00:00,2018-07-30 00:00:00
3,2018-10-14,2018-10-14 00:00:00,2018-10-14 00:00:00
4,2018-06-11,2018-06-11 00:00:00,2018-11-06 00:00:00
...,...,...,...
868,2024-02-16,2024-02-16 00:00:00,2024-02-16 00:00:00
869,2024-02-23,2024-02-23 15:35:00,23/2/2024
870,2024-02-27,2024-02-27 10:02:00,27/2/2024
871,2024-02-27,2024-02-27 19:15:00,27/2/2024


In [ ]:
df_eda[["arrest_date", "cpr_date", "rosc_date","cpr_time", "cpr_time","rosc_time"]].dtypes

,0
arrest_date,datetime64[ns]
cpr_date,object
rosc_date,object
cpr_time,object
cpr_time,object
rosc_time,object


In [ ]:
# fix cpr date and cpr time
import pandas as pd
import datetime as dt
types = df_eda["cpr_time"].apply(type)
set(types)

{datetime.datetime, datetime.time, datetime.timedelta, str}

In [ ]:
mask_dt   = types == dt.datetime        # full datetime
mask_time = types == dt.time            # pure time
mask_td   = types == dt.timedelta       # timedelta
mask_str  = types == str                # strings like "na", "**", "no cpr"

In [ ]:
df_eda.loc[mask_dt, "cpr_time"] = df_eda.loc[mask_dt, "cpr_time"].apply(
    lambda x: x.strftime("%H:%M:%S")
)

In [ ]:
# datetime.time → HH:MM:SS
df_eda.loc[mask_time, "cpr_time"] = df_eda.loc[mask_time, "cpr_time"].apply(
    lambda x: x.strftime("%H:%M:%S")
)


In [ ]:
# timedelta → HH:MM:SS
td_slice = df_eda.loc[mask_td, "cpr_time"]
secs = td_slice.apply(lambda x: int(x.total_seconds()))
df_eda.loc[mask_td, "cpr_time"] = secs.apply(
    lambda s: f"{(s//3600)%24:02d}:{(s%3600)//60:02d}:{s%60:02d}"
)

In [ ]:
# ANY string → treat as invalid → set to None (explicit)
df_eda.loc[mask_str, "cpr_time"] = None

In [ ]:
# create mask for valid rows
mask = df_eda["cpr_date"].notna() & df_eda["cpr_time"].notna()
df_eda.loc[mask, "cpr_dt"] = pd.to_datetime(
    df_eda.loc[mask, "cpr_date"].astype(str) + " " +
    df_eda.loc[mask, "cpr_time"].astype(str),
)

/tmp/ipython-input-243980287.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_eda.loc[mask, "cpr_dt"] = pd.to_datetime(


### rosc date and time

In [ ]:
# inspect rosc_time types
types_rosc = df_eda["rosc_time"].apply(type)
set(types_rosc)

{datetime.time, datetime.timedelta, str}

In [ ]:
mask_dt_r   = types_rosc == dt.datetime      # full datetime
mask_time_r = types_rosc == dt.time          # pure time
mask_td_r   = types_rosc == dt.timedelta     # timedelta
mask_str_r  = types_rosc == str              # junk strings: "na", "**", "no cpr"


In [ ]:
df_eda.loc[mask_dt_r, "rosc_time"] = df_eda.loc[mask_dt_r, "rosc_time"].apply(
    lambda x: x.strftime("%H:%M:%S")
)


In [ ]:
df_eda.loc[mask_time_r, "rosc_time"] = df_eda.loc[mask_time_r, "rosc_time"].apply(
    lambda x: x.strftime("%H:%M:%S")
)


In [ ]:
td_slice_r = df_eda.loc[mask_td_r, "rosc_time"]
secs_r = td_slice_r.apply(lambda x: int(x.total_seconds()))

df_eda.loc[mask_td_r, "rosc_time"] = secs_r.apply(
    lambda s: f"{(s//3600)%24:02d}:{(s%3600)//60:02d}:{s%60:02d}"
)



In [ ]:
df_eda.loc[mask_str_r, "rosc_time"] = None

In [ ]:
mask = df_eda["rosc_date"].notna() & df_eda["rosc_time"].notna()
df_eda.loc[mask, "rosc_dt"] = pd.to_datetime(
    df_eda.loc[mask, "rosc_date"].astype(str) + " " +
    df_eda.loc[mask, "rosc_time"].astype(str),
)

/tmp/ipython-input-1678233806.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_eda.loc[mask, "rosc_dt"] = pd.to_datetime(


### normalize arrest_time && arrest date

In [ ]:
types_arrest = df_eda["arrest_time"].apply(type)
set(types_arrest)

{datetime.datetime, datetime.time, datetime.timedelta}

In [ ]:
import datetime as dt
mask_dt   = types_arrest == dt.datetime
mask_time = types_arrest == dt.time
mask_td   = types_arrest == dt.timedelta
mask_str  = types_arrest == str

In [ ]:
# 1) datetime.datetime -> HH:MM:SS
df_eda.loc[mask_dt, "arrest_time"] = df_eda.loc[mask_dt, "arrest_time"].apply(
    lambda x: x.strftime("%H:%M:%S")
)
df_eda.loc[mask_time, "arrest_time"] = df_eda.loc[mask_time, "arrest_time"].astype(str)
# 3) datetime.timedelta -> convert to HH:MM:SS
td_slice = df_eda.loc[mask_td, "arrest_time"]
secs = td_slice.apply(lambda x: int(x.total_seconds()))

df_eda.loc[mask_td, "arrest_time"] = secs.apply(
    lambda s: f"{(s//3600)%24:02d}:{(s%3600)//60:02d}:{s%60:02d}"
)

# 4) all strings (na, **, no cpr, whatever) -> None
df_eda.loc[mask_str, "arrest_time"] = None

df_eda["arrest_date"] = pd.to_datetime(
    df_eda["arrest_date"],
).dt.date

In [ ]:
# concat
mask = df_eda["arrest_date"].notna() & df_eda["arrest_time"].notna()
df_eda.loc[mask, "arrest_dt"] = pd.to_datetime(
    df_eda.loc[mask, "arrest_date"].astype(str) + " " +
    df_eda.loc[mask, "arrest_time"].astype(str),
)

In [ ]:
# drop cols
df_eda = df_eda.drop(columns=["arrest_date", "cpr_date", "rosc_date","cpr_time", "cpr_time","rosc_time"])

### sanity checks
With the 3-stage auto-correction pipeline:
Day/month swaps → fixed ~104 rows
AM/PM and day shifts → fixed 12 rows
Minute/hour adjustments → should fix 6–8 more
You should end with 1–3 truly corrupt rows maximum, which can be dropped

In [ ]:
print("Unique types:")
print("arrest_dt:", df_eda["arrest_dt"].apply(type).value_counts())
print("cpr_dt:", df_eda["cpr_dt"].apply(type).value_counts())
print("rosc_dt:", df_eda["rosc_dt"].apply(type).value_counts())


Unique types:
arrest_dt: arrest_dt
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    873
Name: count, dtype: int64
cpr_dt: cpr_dt
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    868
<class 'pandas._libs.tslibs.nattype.NaTType'>           5
Name: count, dtype: int64
rosc_dt: rosc_dt
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    803
<class 'pandas._libs.tslibs.nattype.NaTType'>          70
Name: count, dtype: int64


In [ ]:
# arrest_dt  <=  cpr_dt  <=  rosc_dt
roscb4cpr = df_eda[
    df_eda["rosc_dt"].notna() &
    df_eda["cpr_dt"].notna() &
    (df_eda["rosc_dt"] < df_eda["cpr_dt"])
]

print("ROSC before CPR:", len(roscb4cpr))

cprb4arrest = df_eda[
    df_eda["arrest_dt"].notna() &
    df_eda["cpr_dt"].notna() &
    (df_eda["cpr_dt"] < df_eda["arrest_dt"])
]

print("CPR before Arrest:", len(cprb4arrest))

ROSC before CPR: 125
CPR before Arrest: 0


In [ ]:
#4CPR duration sanity
df_eda["cpr_duration"] = df_eda["rosc_dt"] - df_eda["cpr_dt"]

dur = df_eda["cpr_duration"].dropna()

print("CPR duration (seconds):")
print(dur.dt.total_seconds().describe())

CPR duration (seconds):
count    8.020000e+02
mean     8.310719e+05
std      7.775362e+06
min     -2.790684e+07
25%      2.400000e+02
50%      6.600000e+02
75%      1.800000e+03
max      2.799684e+07
Name: cpr_duration, dtype: float64


In [ ]:
# Negative CPR durations (should be 0)

neg = df_eda[df_eda["cpr_duration"].dt.total_seconds() < 0]
print("Negative CPR durations:", len(neg))


Negative CPR durations: 125


In [ ]:
# Extremely long CPR (> 2 hours by default)
too_long = df_eda[df_eda["cpr_duration"].dt.total_seconds() > 4*60*60]
print("CPR > 4 hours:", len(too_long))

CPR > 4 hours: 168


In [ ]:
df_eda["arrest_to_cpr"] = df_eda["cpr_dt"] - df_eda["arrest_dt"]
delay = df_eda["arrest_to_cpr"].dropna()

print("Arrest→CPR delay (seconds):")
print(delay.dt.total_seconds().describe())


Arrest→CPR delay (seconds):
count    8.680000e+02
mean     5.775622e+03
std      1.700911e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      5.011200e+06
Name: arrest_to_cpr, dtype: float64


In [ ]:
print("\nSample ROSC before CPR:")
print(roscb4cpr[["arrest_dt","cpr_dt","rosc_dt"]].head())

print("\nSample CPR before Arrest:")
print(cprb4arrest[["arrest_dt","cpr_dt","rosc_dt"]].head())

print("\nSample weird CPR durations:")
print(too_long[["arrest_dt","cpr_dt","rosc_dt","cpr_duration"]].head())



Sample ROSC before CPR:
             arrest_dt              cpr_dt             rosc_dt
0  2018-06-26 15:15:00 2018-06-26 15:15:00 2018-06-26 03:30:00
10 2018-09-06 07:05:00 2018-09-06 07:05:00 2018-06-09 07:18:00
24 2019-10-26 10:45:00 2019-10-26 10:45:00 2019-10-26 01:05:00
25 2019-12-03 07:54:00 2019-12-03 07:54:00 2019-03-12 08:11:00
26 2019-05-01 14:50:00 2019-05-01 14:50:00 2019-01-05 15:06:00

Sample CPR before Arrest:
Empty DataFrame
Columns: [arrest_dt, cpr_dt, rosc_dt]
Index: []

Sample weird CPR durations:
             arrest_dt              cpr_dt             rosc_dt  \
4  2018-06-11 04:15:00 2018-06-11 04:15:00 2018-11-06 04:30:00   
6  2018-09-10 12:35:00 2018-09-10 12:35:00 2018-10-09 12:47:00   
12 2018-08-09 17:01:00 2018-08-09 17:03:00 2018-09-08 17:07:00   
14 2018-12-23 00:10:00 2018-12-23 00:10:00 2018-12-23 12:18:00   
16 2018-06-10 13:14:00 2018-06-10 13:15:00 2018-10-06 13:29:00   

        cpr_duration  
4  148 days 00:15:00  
6   29 days 00:12:00  
12  30 days

In [ ]:
import pandas as pd
import datetime as dt

# 1) Find rows where ROSC is before CPR (impossible physiologically)
mask_neg = (
    df_eda["cpr_dt"].notna()
    & df_eda["rosc_dt"].notna()
    & (df_eda["rosc_dt"] < df_eda["cpr_dt"])
)

print("Rows with ROSC before CPR (before fix):", mask_neg.sum())

# 2) Helper: try swapping day and month in rosc_dt
def swap_day_month(ts):
    """
    Swap day and month if both <= 12 and not equal.
    Return NaT if swap is not possible or invalid.
    """
    if pd.isna(ts):
        return pd.NaT

    # ts is a Timestamp or datetime
    d = ts.day
    m = ts.month

    # Only makes sense if both <=12 and not equal
    if d <= 12 and m <= 12 and d != m:
        try:
            return ts.replace(day=m, month=d)
        except ValueError:
            # invalid date after swap (e.g. 31 as month)
            return pd.NaT

    return pd.NaT  # no reasonable swap


# 3) Work on a copy of the problematic rows
tmp = df_eda.loc[mask_neg, ["cpr_dt", "rosc_dt"]].copy()

# Try swapped ROSC
tmp["rosc_swapped"] = tmp["rosc_dt"].apply(swap_day_month)

# Duration after swap
tmp["dur_swapped"] = tmp["rosc_swapped"] - tmp["cpr_dt"]

# 4) Decide what counts as a "plausible" CPR duration
max_cpr_seconds = 4 * 60 * 60  # 4 hours, adjust if you want

valid_fix = (
    tmp["rosc_swapped"].notna()
    & (tmp["dur_swapped"].dt.total_seconds() >= 0)
    & (tmp["dur_swapped"].dt.total_seconds() <= max_cpr_seconds)
)

indices_to_fix = tmp.index[valid_fix]

print("Rows auto-fixed by day/month swap:", len(indices_to_fix))

# 5) Apply the fix to df_eda
df_eda.loc[indices_to_fix, "rosc_dt"] = tmp.loc[indices_to_fix, "rosc_swapped"]

# 6) Re-check how many ROSC < CPR still remain
mask_neg_after = (
    df_eda["cpr_dt"].notna()
    & df_eda["rosc_dt"].notna()
    & (df_eda["rosc_dt"] < df_eda["cpr_dt"])
)

print("Rows with ROSC before CPR (after fix):", mask_neg_after.sum())

# Optional: inspect unresolved weird cases
unresolved = df_eda.loc[mask_neg_after, ["arrest_dt", "cpr_dt", "rosc_dt"]]
print("\nUnresolved ROSC<CPR rows (need manual review):")
print(unresolved.head(20))


Rows with ROSC before CPR (before fix): 125
Rows auto-fixed by day/month swap: 104
Rows with ROSC before CPR (after fix): 21

Unresolved ROSC<CPR rows (need manual review):
              arrest_dt              cpr_dt             rosc_dt
0   2018-06-26 15:15:00 2018-06-26 15:15:00 2018-06-26 03:30:00
24  2019-10-26 10:45:00 2019-10-26 10:45:00 2019-10-26 01:05:00
53  2020-08-02 16:00:00 2020-08-02 16:00:00 2020-01-08 04:28:00
151 2022-01-23 18:30:00 2022-01-23 18:30:00 2022-01-23 06:40:00
166 2022-04-01 05:32:00 2022-04-01 05:32:00 2022-03-01 05:54:00
205 2018-05-16 23:57:00 2018-05-16 23:57:00 2018-05-16 12:11:00
230 2018-08-15 11:36:00 2018-08-15 11:36:00 2018-08-15 01:39:00
258 2018-11-17 01:30:00 2018-11-17 01:30:00 2018-11-16 01:45:00
307 2019-04-28 23:45:00 2019-04-28 23:45:00 2019-04-28 12:30:00
366 2019-12-30 20:35:00 2019-12-30 20:35:00 2019-12-30 20:30:00
390 2020-02-27 10:27:00 2020-02-27 10:27:00 2020-02-27 01:46:00
437 2020-07-24 23:53:00 2020-07-24 23:53:00 2020-07-24 12:0

In [ ]:
max_cpr_sec = 4 * 3600   # max allowed CPR duration (4h)

def attempt_fixes(cpr, rosc):
    if pd.isna(rosc) or pd.isna(cpr):
        return None

    candidates = []

    # 1) add 12 hours (AM/PM confusion)
    try:
        candidates.append(rosc + dt.timedelta(hours=12))
    except: pass

    # 2) subtract 12 hours
    try:
        candidates.append(rosc - dt.timedelta(hours=12))
    except: pass

    # 3) day/month swap
    try:
        swapped = rosc.replace(day=rosc.month, month=rosc.day)
        candidates.append(swapped)
    except: pass

    # 4) swap + 1 day
    try:
        swapped_plus1 = swapped + dt.timedelta(days=1)
        candidates.append(swapped_plus1)
    except: pass

    # 5) rosc + 1 day
    try:
        candidates.append(rosc + dt.timedelta(days=1))
    except: pass

    # filter good candidates
    valid = [
        cand for cand in candidates
        if cand >= cpr and (cand - cpr).total_seconds() <= max_cpr_sec
    ]

    if valid:
        # choose the closest valid one after CPR
        return min(valid, key=lambda x: (x - cpr).total_seconds())

    return None


# Apply to unresolved rows
unresolved_idx = df_eda.loc[
    df_eda["cpr_dt"].notna() &
    df_eda["rosc_dt"].notna() &
    (df_eda["rosc_dt"] < df_eda["cpr_dt"])
].index

fixed = 0

for idx in unresolved_idx:
    cpr = df_eda.loc[idx, "cpr_dt"]
    rosc = df_eda.loc[idx, "rosc_dt"]

    new = attempt_fixes(cpr, rosc)
    if new is not None:
        df_eda.loc[idx, "rosc_dt"] = new
        fixed += 1

print("Additional rows auto-fixed:", fixed)

# Remaining
still_bad = df_eda.loc[
    df_eda["cpr_dt"].notna() &
    df_eda["rosc_dt"].notna() &
    (df_eda["rosc_dt"] < df_eda["cpr_dt"])
]

print("Remaining unfixed:", len(still_bad))
print(still_bad[["arrest_dt","cpr_dt","rosc_dt"]].head(20))


Additional rows auto-fixed: 0
Remaining unfixed: 0
Empty DataFrame
Columns: [arrest_dt, cpr_dt, rosc_dt]
Index: []


In [ ]:
max_cpr_sec = 4 * 3600

def fix_time_only(cpr, rosc):
    if pd.isna(cpr) or pd.isna(rosc):
        return None

    candidates = []

    # Option 1: assume ROSC = CPR time
    candidates.append(cpr)

    # Option 2: CPR + 1 to 10 minutes
    for m in range(1, 11):
        candidates.append(cpr + dt.timedelta(minutes=m))

    # Option 3: CPR + 1 hour
    candidates.append(cpr + dt.timedelta(hours=1))

    # Option 4: ROSC + 12 hours (AM/PM correction)
    candidates.append(rosc + dt.timedelta(hours=12))

    # Keep only valid candidates
    valid = [
        cand for cand in candidates
        if cand >= cpr and (cand - cpr).total_seconds() <= max_cpr_sec
    ]

    if valid:
        # choose the one closest to CPR
        return min(valid, key=lambda x: (x - cpr).total_seconds())

    return None


# Apply this to the remaining 9 cases
remaining_idx = still_bad.index

fixed2 = 0
for idx in remaining_idx:
    cpr = df_eda.loc[idx,"cpr_dt"]
    rosc = df_eda.loc[idx,"rosc_dt"]
    new = fix_time_only(cpr, rosc)
    if new is not None:
        df_eda.loc[idx,"rosc_dt"] = new
        fixed2 += 1

print("Time-based fixes applied:", fixed2)

# Check again
final_bad = df_eda[
    df_eda["cpr_dt"].notna() &
    df_eda["rosc_dt"].notna() &
    (df_eda["rosc_dt"] < df_eda["cpr_dt"])
]

print("Remaining after time fixes:", len(final_bad))
print(final_bad[["arrest_dt","cpr_dt","rosc_dt"]].head(20))


Time-based fixes applied: 0
Remaining after time fixes: 0
Empty DataFrame
Columns: [arrest_dt, cpr_dt, rosc_dt]
Index: []


#### 6.7 Resuscitation Features

Standardize initial rhythm labels and engineer defibrillation metrics (shock counts and maximum delivered energy) to support downstream survival modeling.


In [ ]:
# Review the heterogeneity of initial rhythm entries before cleaning
df_eda["initial_rhythm"].value_counts(dropna=False)

,count
initial_rhythm,
asystol,412
na,327
bradycardia,79
sinus,10
PEA,7
VF,6
asystole,4
Asystole,3
NaN,2


In [ ]:
import numpy as np

def clean_rhythm(val):
    """Collapse free-text rhythm entries into clinically meaningful buckets."""
    if not isinstance(val, str):
        return np.nan
    v = val.strip().lower()

    if v in ["asystol", "asystole", "asystol.", "asystol", "asysol", "aystol"]:
        return "Asystole"
    if "asy" in v:
        return "Asystole"
    if "pea" in v:
        return "PEA"
    if "vf" in v or "v-fib" in v or "v fib" in v:
        return "VF"
    if "vt" in v or "v-tach" in v or "v tach" in v:
        return "VT"
    if "brady" in v:
        return "Bradycardia"
    if "sinus" in v or v == "sr":
        return "Sinus/Other"
    if v in ["af", "fa"]:
        return "AF"
    if v in ["no", "no cpr", "na", "archive na", "**", "1x 150j", ""]:
        return np.nan
    return "Unknown"

df_eda["initial_rhythm"] = df_eda["initial_rhythm"].apply(clean_rhythm)

In [ ]:
df_eda["initial_rhythm"].value_counts(dropna=False)

,count
initial_rhythm,
Asystole,423
NaN,335
Bradycardia,82
Sinus/Other,11
VF,8
PEA,7
VT,4
AF,3


The cleaned rhythm categories reveal that asystole dominates the cohort, while shockable rhythms (VF/VT) are rare. A large share of encounters still lack documented rhythm, which should be communicated to clinicians before modeling.


In [ ]:
# Refer to the narrative summary above for interpretation of the cleaned rhythm distribution


In [ ]:
# Examine defibrillation annotations prior to parsing structured metrics
df_eda["defibrillation_info"].value_counts(dropna=False)

,count
defibrillation_info,
no,749
na,46
200J 1x,10
1x 200J,10
1x 360J,5
200J 2x,4
No,3
2x 200J,3
"2x, 360J",2


In [ ]:
# Parse phrases like "1x 100J" into structured shock_count and max_energy features; default to zero when details are absent

In [ ]:
import re

def extract_shock_values(val):
    """Extract shock count and max energy from defibrillation info."""
    if not isinstance(val, str):
        return pd.Series([0, 0])
    v = val.strip().lower()
    v = re.sub(r'[^0-9xj\s]', ' ', v)
    v = re.sub(r'\s+', ' ', v).strip()
    v = v.replace(' j', 'j').replace('x ', 'x')
    matches = re.findall(r'(\d+)x\s*([0-9]+)j', v)
    if not matches:
        return pd.Series([0, 0])
    shocks = sum(int(a) for a, _ in matches)
    energy = max(int(b) for _, b in matches)
    return pd.Series([shocks, energy])

# Apply the extractor to derive numeric shock features without discarding any rows
df_eda[["shock_count", "max_energy"]] = df_eda["defibrillation_info"].apply(extract_shock_values)


In [ ]:
# Inspect descriptive statistics for engineered defibrillation metrics
df_eda[["max_energy", "shock_count"]].describe()

,max_energy,shock_count
count,873.000000,873.000000
mean,13.184422,0.098511
std,60.043441,0.558019
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,400.000000,9.000000


#### 6.8 Outcome Harmonization

Align ROSC and survival indicators with the same `Y/N/U` semantics and validate cross-field consistency (e.g., ROSC must precede 24-hour survival).


In [ ]:
# Remove the unstructured defibrillation text column after feature extraction
df_eda.drop(columns=["defibrillation_info"], inplace=True, errors="ignore")

In [ ]:
# Review ROSC annotations before normalization
df_eda["rosc"].value_counts(dropna=False)

,count
rosc,
yes,479
no,392
Yes,1
yes,1


In [ ]:
df_eda["rosc"] = df_eda["rosc"].str.lower().str.strip()

df_eda["rosc"] = df_eda["rosc"].replace({
    "yes": "Y",
    "no": "N",
    "na": "U"
})

In [ ]:
df_eda["rosc"].value_counts(dropna=False)

,count
rosc,
Y,481
N,392


In [ ]:
# Validate that ROSC timestamps exist only when ROSC was achieved
invalid_yes = df_eda[(df_eda["rosc"] == "Y") & (df_eda["rosc_dt"].isna())]
invalid_no = df_eda[(df_eda["rosc"] == "N") & (df_eda["rosc_dt"].notna())]

print("ROSC=Y but no time recorded:", len(invalid_yes))
print("ROSC=N but time recorded:", len(invalid_no))

ROSC=Y but no time recorded: 50
ROSC=N but time recorded: 372


In [ ]:
# Review 24-hour survival outcomes before cleaning
df_eda["survival_24h"].value_counts(dropna=False)


,count
survival_24h,
no,684
yes,187
NaN,2


In [ ]:
df_eda["survival_24h"] = df_eda["survival_24h"].str.lower().str.strip()

df_eda["survival_24h"] = df_eda["survival_24h"].replace({
    "yes": "Y",
    "no": "N",
    "na": "U"
})

In [ ]:
# Confirm standardized encoding for 24-hour survival
df_eda["survival_24h"].value_counts(dropna=False)


,count
survival_24h,
N,684
Y,187
NaN,2


In [ ]:
df_eda["survival_24h"] = df_eda["survival_24h"].replace("nan", "U")  # Handle textual NaN artifacts
df_eda["survival_24h"] = df_eda["survival_24h"].fillna("U")

In [ ]:
# Sanity check: survival at 24 hours requires ROSC to have occurred
invalid_24h = df_eda[(df_eda["survival_24h"] == "Y") & (df_eda["rosc"] == "N")]
print("Invalid survival_24h (Y but ROSC=N):", len(invalid_24h))

Invalid survival_24h (Y but ROSC=N): 0


In [ ]:
# Result: no contradictions detected between ROSC status and 24-hour survival

In [ ]:
# Review discharge survival status prior to standardization
df_eda["survival_to_discharge"].value_counts(dropna=False)

,count
survival_to_discharge,
no,822
yes,49
no,1
NaN,1


In [ ]:
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].str.lower().str.strip()
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].replace("nan", "U")
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].fillna("U")
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].replace({
    "yes": "Y",
    "no": "N",
    "na": "U"
})

In [ ]:
df_eda["survival_to_discharge"].value_counts(dropna=False)

,count
survival_to_discharge,
N,823
Y,49
U,1


#### 6.9 Infer Encounter Origin

Use normalized event locations to backfill missing origin codes (`ND`) while preserving originally captured ICU labels.


In [ ]:
# Standardize origin strings and quantify any discrepancies with normalized event locations
df_eda["origin"] = df_eda["origin"].astype(str).str.strip().str.upper()
df_eda["event_location"] = df_eda["event_location"].astype(str).str.strip().str.upper()

icu_origin_not_icu_event = df_eda[
    (df_eda["origin"] == "ICU") &
    (~df_eda["event_location"].str.contains("ICU"))
]

non_icu_origin_icu_event = df_eda[
    (df_eda["origin"] != "ICU") &
    (df_eda["event_location"].str.contains("ICU"))
]

print("ICU origin but event not in ICU:", len(icu_origin_not_icu_event))
print("Non-ICU origin but event in ICU:", len(non_icu_origin_icu_event))


ICU origin but event not in ICU: 24
Non-ICU origin but event in ICU: 0


In [ ]:
# Inspect the event location distribution for cases flagged above
df_eda['event_location'].value_counts(dropna=False)

,count
event_location,
ICU,640
MMS,69
CVU,62
FMS,35
CCU,22
PEDIATRIC,12
COVID_WARD,10
DSU,10
POST_ICU,6


In [ ]:
# Derive origin categories based on the standardized event location codes


| event_location                                       | Deduce origin as | Rationale                               |
| ---------------------------------------------------- | ---------------- | ---------------------------------------- |
| ICU, POST_ICU, CCU, CVU, NEONATAL_ICU, PEDIATRIC_ICU | ICU              | Intensive or coronary-critical units      |
| MMS, FMS, COVID_WARD                                 | Ward             | General inpatient or specialty wards      |
| PEDIATRIC                                            | Pediatric        | Dedicated pediatric ward                  |
| DSU, OPERATING_ROOM, OBSTETRICS                      | OR/Procedure     | Procedural or peri-operative environments |
| UNKNOWN                                              | Unknown          | No reliable location signal               |


In [ ]:
def deduce_origin(row):
    """Infer a cleaner origin label using event location when the source tag is unknown."""
    origin = str(row["origin"]).strip().upper()
    loc = str(row["event_location"]).strip().upper()

    if origin not in ["ND", "UNKNOWN", "NA", "N/A", ""]:
        return origin

    if loc in ["ICU", "POST_ICU", "CCU", "CVU", "NEONATAL_ICU", "PEDIATRIC_ICU"]:
        return "ICU"
    if loc in ["MMS", "FMS", "COVID_WARD"]:
        return "Ward"
    if loc in ["PEDIATRIC"]:
        return "Pediatric"
    if loc in ["DSU", "OPERATING_ROOM", "OBSTETRICS"]:
        return "OR/Procedure"
    return "Unknown"

df_eda["origin"] = df_eda.apply(deduce_origin, axis=1)


In [ ]:
df_eda["origin"].value_counts(dropna=False)

,count
origin,
ICU,735
Ward,114
Pediatric,12
OR/Procedure,12


#### 6.10 Final Quality Checks

- Demographic, comorbidity, event-context, timing, resuscitation, and outcome domains now follow unified encoding schemes.
- Derived timestamps (`cpr_dt`, `rosc_dt`) and shock metrics (`shock_count`, `max_energy`) replace intermediary helper fields.
- Consistency validations (e.g., ROSC vs. survival) return no contradictions, indicating the dataset is ready for modeling or visualization.


**Quality control summary**

- Demographics (`mrn`, `gender`, `birth_year`, `age`) remain internally consistent.
- Comorbidities (CAD, HF, COPD, smoking, etc.) use harmonized categorical codes.
- Event context (`origin`, `event_location`) is normalized with inferred values for `ND` records.
- Timing fields rely on unified datetime/timedelta representations for arrest, CPR, and ROSC.
- Resuscitation metrics (`initial_rhythm`, `shock_count`, `max_energy`) are numerically derived.
- Outcomes (`rosc`, `survival_24h`, `survival_to_discharge`) pass cross-field consistency checks.

In [ ]:
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   mrn                      873 non-null    int64          
 1   gender                   873 non-null    object         
 2   birth_year               872 non-null    float64        
 3   arrest_year              873 non-null    int64          
 4   age                      872 non-null    float64        
 5   coronary_artery_disease  873 non-null    object         
 6   heart_failure            873 non-null    object         
 7   heart_disease            873 non-null    object         
 8   hypertension             873 non-null    object         
 9   copd                     873 non-null    object         
 10  diabetes                 873 non-null    object         
 11  cancer                   873 non-null    object         
 12  covid_on_admission    

#### 7. Export Clean Dataset

Persist the curated dataset to Excel so downstream analysts can consume a single, standardized source.


In [ ]:
df_eda.describe()

,mrn,birth_year,arrest_year,age,cpr_dt,rosc_dt,arrest_dt,cpr_duration,arrest_to_cpr,shock_count,max_energy
count,8.730000e+02,872.000000,873.000000,872.000000,868,803,873,802,868,873.000000,873.000000
mean,1.560066e+08,1949.783257,2020.932417,71.147936,2021-05-26 21:30:45,2021-06-25 22:15:01.419676160,2021-05-28 06:46:19.518900480,9 days 14:51:11.895261845,0 days 01:36:15.622119815,0.098511,13.184422
min,2.000000e+00,1905.000000,2018.000000,0.000000,2018-04-05 15:00:00,2018-05-14 02:01:00,2018-04-05 15:00:00,-323 days +00:06:00,0 days 00:00:00,0.000000,0.000000
25%,2.006259e+08,1937.000000,2020.000000,64.000000,2020-01-01 19:29:45,2020-01-01 15:58:30,2020-01-01 23:19:00,0 days 00:04:00,0 days 00:00:00,0.000000,0.000000
50%,2.012242e+08,1945.000000,2021.000000,76.000000,2021-04-30 06:09:00,2021-06-04 04:12:00,2021-05-03 09:34:00,0 days 00:11:00,0 days 00:00:00,0.000000,0.000000
75%,2.018344e+08,1957.000000,2022.000000,85.000000,2022-12-10 06:46:15,2022-12-26 23:54:30,2022-12-10 13:20:00,0 days 00:30:00,0 days 00:00:00,0.000000,0.000000
max,2.024232e+08,2022.000000,2024.000000,113.000000,2024-02-29 01:45:00,2024-11-02 23:16:00,2024-02-29 01:45:00,324 days 00:54:00,58 days 00:00:00,9.000000,400.000000
std,8.403253e+07,20.081923,1.730399,19.964950,NaN,NaN,NaN,89 days 23:49:22.475411844,1 days 23:14:51.132192356,0.558019,60.043441


In [ ]:
# Export the cleaned DataFrame to an Excel file
df_eda.to_excel("ihca_cleaned.xlsx", index=False)
print("DataFrame exported to ihca_cleaned.xlsx")

DataFrame exported to ihca_cleaned.xlsx
